In [396]:
# Definição das classes Disciplina, Curso e Unidade
class Disciplina:
    def __init__(self, codigo, nome, creditos_aula, creditos_trabalho,
                 carga_horaria, carga_estagio, carga_praticas, carga_atividades):
        self.codigo = codigo
        self.nome = nome
        self.creditos_aula = creditos_aula
        self.creditos_trabalho = creditos_trabalho
        self.carga_horaria = carga_horaria
        self.carga_estagio = carga_estagio
        self.carga_praticas = carga_praticas
        self.carga_atividades = carga_atividades

    def mostrar(self):
        print(f"  {self.codigo} - {self.nome}")
        print(f"    Créditos Aula: {self.creditos_aula}")
        print(f"    Créditos Trabalho: {self.creditos_trabalho}")
        print(f"    CH: {self.carga_horaria}, Estágio: {self.carga_estagio}, Práticas: {self.carga_praticas}, Atividades: {self.carga_atividades}")
        
    def __repr__(self):
        return f"{self.codigo} - {self.nome}"


class Curso:
    def __init__(self, nome, unidade, duracao_ideal, duracao_minima, duracao_maxima):
        self.nome = nome
        self.unidade = unidade
        self.duracao_ideal = duracao_ideal
        self.duracao_minima = duracao_minima
        self.duracao_maxima = duracao_maxima
        self.obrigatorias = []
        self.optativas_livres = []
        self.optativas_eletivas = []

    def adicionar_disciplina(self, disciplina, tipo):
        if tipo == "obrigatoria":
            self.obrigatorias.append(disciplina)
        elif tipo == "optativa_livre":
            self.optativas_livres.append(disciplina)
        elif tipo == "optativa_eletiva":
            self.optativas_eletivas.append(disciplina)

    def mostrar(self):
        print(f"Curso: {self.nome}")
        print(f"  Unidade: {self.unidade}")
        print(f"  Duração Ideal: {self.duracao_ideal} | Mínima: {self.duracao_minima} | Máxima: {self.duracao_maxima}")
        print("\n  Disciplinas Obrigatórias:")
        for d in self.obrigatorias:
            d.mostrar()
        print("\n  Disciplinas Optativas Eletivas:")
        for d in self.optativas_eletivas:
            d.mostrar()
        print("\n  Disciplinas Optativas Livres:")
        for d in self.optativas_livres:
            d.mostrar()

    def __repr__(self):
        return f"{self.nome} ({self.unidade})"


class Unidade:
    def __init__(self, nome):
        self.nome = nome
        self.cursos = []

    def adicionar_curso(self, curso):
        self.cursos.append(curso)

    def mostrar(self):
        print(f"Unidade: {self.nome}")
        print(f"Total de cursos: {len(self.cursos)}")
        for curso in self.cursos:
            print("\n---------------------------")
            curso.mostrar()
            
    def __repr__(self):
        return f"Unidade: {self.nome} - {len(self.cursos)} cursos"

In [517]:
### BIBLIOTECAS UTILIZADAS: SELENIUM; BEAUTIFULSOUP4; TIME
### IMPORTANTE: PRECISA BAIXAR CADA BIBLIOTECA UTILIZANDO pip install nomeBiblioteca no terminal
### ex.: pip install selenium

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import time

# Abrir o navegador
navegador = webdriver.Chrome()
navegador.get("https://uspdigital.usp.br/jupiterweb/jupCarreira.jsp?codmnu=8275")

In [529]:
# aqui ele só seleciona uma unidade e depois um curso
# eu so fiz 1 seleção para testar a extração dos dados
# precisa montar o loop onde ele percorre cada um

Select(navegador.find_element(By.ID,"comboUnidade")).select_by_index(3)
time.sleep(0.2)

Select(navegador.find_element(By.ID,"comboCurso")).select_by_index(1)
time.sleep(0.2)

navegador.find_element(By.ID,"enviar").click()
time.sleep(0.2)

navegador.find_element(By.ID,"step4-tab").click()

In [530]:
# Chama o beautifulsoup pra poder extrair os dados da pagina

page_source = navegador.page_source

soup = BeautifulSoup(page_source, 'html.parser')

unidades = []

In [531]:
# extrai as info do curso
nomeUnidade = soup.find_all('span', class_='unidade')[0].text
nomeCurso = soup.find_all('span', class_='curso')[0].text
duracaoIdeal = soup.find_all('span', class_='duridlhab')[0].text
duracaoMinima = soup.find_all('span', class_='durminhab')[0].text
duracaoMaxima = soup.find_all('span', class_='durmaxhab')[0].text

unidade = Unidade(nomeUnidade) # cria a unidade
curso = Curso(nomeCurso, nomeUnidade, duracaoIdeal, duracaoMinima, duracaoMaxima) # cria o curso

In [532]:
# extrai as info das disciplinas

disciplinas_extraidas = soup.find(id="gradeCurricular").find_all("td")
linhas = [disciplinas_extraidas[i].text for i in range(len(disciplinas_extraidas))]

i=0
disciplinas=[]
tipo = None
while(i < len(cursos)):
    if(linhas[i]=="Disciplinas Obrigatórias"):
        tipo = "obrigatoria"
        i=i+1
    elif(linhas[i]=="Disciplinas Optativas Eletivas"):
        tipo = "optativa_eletiva"
        i=i+1
    elif(linhas[i]=="Disciplinas Optativas Livres"):
        tipo = "optativa_livre"
        i=i+1
    elif "Semestre" in linhas[i]:
        i=i+7
    else: #rapa o curso
        disciplina = Disciplina(linhas[i], linhas[i+1], linhas[i+2], linhas[i+3], linhas[i+4], linhas[i+5], linhas[i+6], linhas[i+7])
        disciplinas.append((disciplina, tipo))
        i = i+8

# adiciona todas as disciplinas ao curso
for disciplina, tipo in disciplinas:
    curso.adicionar_disciplina(disciplina, tipo)

# adiciona todas as unidades ao curso
unidade.adicionar_curso(curso)

unidades.append(unidade)

[Unidade: Escola de Educação Física e Esporte - ( EEFE ) - 1 cursos]
